In [1]:
# ATTRIBUTES CLASSIFIER

# Imports and Setup
import torch
from torchvision import transforms
from PIL import Image
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm
import faiss
from training.train_triplet import TripletModel


# Paths
root = Path(r"C:\Users\dilku\deepfashion-recsys")
data_path = root / "data" / "deepfashion_index.csv"
ckpt_path = root / "checkpoints" / "debug-epoch=00-train_loss=0.000-v1.ckpt"
export_dir = root / "export"
export_dir.mkdir(parents=True, exist_ok=True)

print("Paths set up")

Paths set up


In [7]:
import sys
from pathlib import Path

# Add src folder to path
sys.path.append(str(Path(r"C:\Users\dilku\deepfashion-recsys\src")))
print("Path configured successfully")

# Import TripletModel
from training.train_triplet import TripletModel
print("TripletModel imported successfully")


from pathlib import Path

ckpt_dir = Path(r"C:\Users\dilku\deepfashion-recsys\checkpoints")
ckpt_path = sorted(ckpt_dir.glob("*.ckpt"), key=lambda p: p.stat().st_mtime, reverse=True)[0]


# Load the fine-tuned TripletModel
from training.train_triplet import TripletModel 

model = TripletModel.load_from_checkpoint(str(ckpt_path))
model.eval()
model.to("cpu")

print("Fine-tuned model loaded successfully")

Path configured successfully
TripletModel imported successfully
Fine-tuned model loaded successfully


In [8]:
# Preparing DataFrames and Transforms
df = pd.read_csv(data_path)
gallery_df = df[df["split"].str.contains("gallery", case=False)]
query_df   = df[df["split"].str.contains("query", case=False)]

print(f"📂 Gallery: {len(gallery_df)} images")
print(f"📂 Query:   {len(query_df)} images")

img_tfms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

📂 Gallery: 12612 images
📂 Query:   14218 images


In [9]:
# Extract embeddings using the fine-tuned model
@torch.no_grad()
def extract_embeddings(image_paths, model):
    embs = []
    for path in tqdm(image_paths, desc="Extracting embeddings"):
        img = Image.open(path).convert("RGB")
        x = img_tfms(img).unsqueeze(0).to("cpu")
        feat = model(x).cpu().numpy()
        embs.append(feat)
    return np.concatenate(embs, axis=0)

gallery_embs = extract_embeddings(gallery_df["image_path"].tolist(), model)
query_embs   = extract_embeddings(query_df["image_path"].tolist(), model)

np.save(export_dir / "gallery_embs_finetuned.npy", gallery_embs)
np.save(export_dir / "query_embs_finetuned.npy", query_embs)

print("Embeddings generated and saved")

Extracting embeddings: 100%|█████████████████████████████████████████████████████| 14218/14218 [21:10<00:00, 11.19it/s]


Embeddings generated and saved


In [10]:
# Build FAISS index for fine-tuned embeddings
dim = gallery_embs.shape[1]
index = faiss.IndexFlatIP(dim)

faiss.normalize_L2(gallery_embs)
index.add(gallery_embs.astype("float32"))

faiss.write_index(index, str(export_dir / "faiss_gallery_finetuned.index"))
print("Finetuned FAISS index saved")

Finetuned FAISS index saved


In [11]:
# Evaluate Recall@K for Query→Gallery retrieval
def recall_at_k(query_embs, gallery_embs, query_labels, gallery_labels, K=[1,5,10,20]):
    faiss.normalize_L2(query_embs)
    D, I = index.search(query_embs.astype("float32"), max(K))
    recalls = {}
    for k in K:
        matches = (gallery_labels[I[:, :k]] == query_labels[:, None])
        recalls[k] = (matches.any(axis=1).mean()) * 100
    return recalls

recalls = recall_at_k(
    query_embs,
    gallery_embs,
    query_df["item_id"].to_numpy(),
    gallery_df["item_id"].to_numpy()
)

print("Recall Scores:")
for k, v in recalls.items():
    print(f"Recall@{k}: {v:.2f}%")

Recall Scores:
Recall@1: 42.82%
Recall@5: 62.51%
Recall@10: 69.91%
Recall@20: 77.11%


In [12]:
print("Model fine-tuning and retrieval evaluation completed successfully.")
print("Generated files:")
print(" - fine_tuned_gallery_embs.npy")
print(" - fine_tuned_query_embs.npy")
print(" - faiss_gallery_finetuned.index")
print("\nNext steps:")
print(" - Compare these Recall@K results with the baseline model to measure improvement.")
print(" - Document performance changes in the README (e.g., Recall@10 increased from 88% → 94%).")

Model fine-tuning and retrieval evaluation completed successfully.
Generated files:
 - fine_tuned_gallery_embs.npy
 - fine_tuned_query_embs.npy
 - faiss_gallery_finetuned.index

Next steps:
 - Compare these Recall@K results with the baseline model to measure improvement.
 - Document performance changes in the README (e.g., Recall@10 increased from 88% → 94%).
